# Maximum Likelihooh Amplitude Estimation (MLAE) module

Present notebook reviews the **Maximum Likelihood Amplitude Estimation** algorithm (**MLAE**) which was implemented into the module *maximum_likelihood_ae* of package *AE* of library *QQuantLib* (**QQuantLib/AE/maximum_likelihood_ae.py**). 

Inside this module we have implemented the **MLAE** as a python class.

Present notebook and module are based on the following references:

* *Suzuki, Y., Uno, S., Raymond, R., Tanaka, T., Onodera, T., & Yamamoto, N.*. Amplitude estimation without phase estimation. Quantum Information Processing, 19(2), 2020. https://arxiv.org/abs/1904.10246

* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

In [ ]:
#For transform bits to int
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$


In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle --svg

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. MLAE algorithm step by step.

The problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $\sqrt{a}$. We can define an associated angle to $\sqrt{a}$ as $\sin^2{\theta} = a$, and the problem is thus rewritten as:
$$\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,$$


We have implemented and python class called **MLAE** into the **QQuantLib/AE/maximum_likelihood_ae** module that allows us implement the **MLAE** algorithm. In this section we are going to describe the class step by step and explain the basics of the **MLAE** algorithm.

In [ ]:
#import the class
from QQuantLib.AE.maximum_likelihood_ae import MLAE

For creating the corresponding object from class **MLAE** following mandatory arguments should be provided:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binnary representation as a python list
3. index: list of the qbits affected by the Grover operator.

The MLAE class creates the Grover operator using the function *grover* from module **QQuantLib/AA/amplitude_amplification**. So the arguments of the **MLAE** class are similar to the arguments of this function (see notebook **02_AmplitudeAmplification_Operators.ipynb** for a detailed explanation)

Additionally a dictionary with other arguments to configure the algorithm can be provided. Keys for the dictionary can be:
* qpu: QLM solver tha will be used (if not provide PyLinalg will be used)
* delta: float for expliciting the tolerance for avoiding divison by zero warnings
* optimizer:  optimizer for solving the optimization problem of the MLAE algorithm (see below)
* schedule: scheduler for applying the grover operator in the MLAE algorithm (see below)
* mcz_qlm: for using QLM multi-controlled Z gate (True, default) or using multiplexor implementation (False)

### 2.1 Creating object from MLAE class

For showing how our class and the algorithm works, we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sin(\theta)|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \cos(\theta)|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
The target state, in this case is $|1\rangle$. It's binary representation is $001$. This has to be passed to the target variable as a list. Moreover we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register. So mandatory arguments will be:
* oracle: QLM AbstractGate
* target: [0, 0, 1]
* index: [0, 1, 2]

Additionally we will provide the qlm solver into the *qpu* key of an input python dictionary

In [ ]:
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True
}
target = [0,0,1]
index = [0,1,2]
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

### 2.2 Application of the grover operator

The foundations of any amplitude estimation algorithm is the grover operator $\mathcal{Q}$. We recall that the grover operator has the following effect over our state $|\Psi\rangle$:

$$\mathcal{G}^{m}|\Psi\rangle = |\Psi \rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$
for more information about the grover operator and the amplitude amplification algorithm check the notebook **02_AmplitudeAmplification_Operators.ipynb**.

The process of generating the corresponding grover oracle is handled automatically in the class when we instance it. To see the grover operator in action we can call the method *run_step* of the class. The input of the method is:

* m_k: the number of times the Grover-like operator will be applied.
* n_k : number of shots.

The method returns:
* h_k: number of positive outcomes

In [ ]:
m_k = 3
n_k = 100
h_k, circuit = mlae.run_step(m_k,n_k)
print('h_k: '+str(h_k))

In [ ]:
%qatdisplay circuit --depth  --svg

From the number of positive outcomes it is straightforward to estimate the probability of getting the state $|1\rangle$ (the positive outcome):
$$ \sin^2\left((2m_k+1)\hat{\theta}\right) = \dfrac{h_k}{n_k} = \hat{p}(|\Psi_0\rangle)\approx p(|\Psi_0\rangle).$$
Here we use the $\hat{}$ to distinguish between the estimated values from the real ones.
So we have a first estimation of our target amplitude:

In [ ]:
angle_estimation = np.arcsin(np.sqrt(h_k/n_k))/(2*m_k+1)
estimation = np.sin(angle_estimation)**2

In [ ]:
print("First estimation: ",estimation)
print("Exact probability: ",probability[bitfield_to_int(target)])

The question now is, ¿how can we consistently improve this result?

### 2.3 The Likelihood

To improve our results we will throw different experiments and combine its information into a single result. To do so we will use the likelihood. First, what is the **likelihood**?

In general, the likelihood $L(a|b)$ is the probability of obtaining $b$ conditioned to $a$, that is $L(a|b) = p(b|a)$.

In our specific case, given the result of an experiment $h_k$ we want to know the probability of an angle $\theta$ being the one that has generated it:
$$p(\theta|h_k).$$
From all the possible values of theta we will propose as a solution the one with most probability.

As computing $p(\theta|h_k)$ is not completely straightforward we will compute instead $p(h_k|\theta)$ which is proportional to the other one by Baye's theorem. The associated likelihood is $L(\theta|h_k) = p(h_k|\theta)$. The value of this likelihood is:

$$L(\theta|h_k) = \sin^2\left((2m_k+1)\theta\right)^{h_k}\cos^2\left((2m_k+1)\theta\right)^{n_k-h_k}.$$

This is because each measurment is independent of the other measurements, the probability of obtaining the state $|1\rangle$ is given by $\sin^2\left((2m_k+1)\theta\right)$  and the probability of the state $|0\rangle$ is given by $\cos^2\left((2m_k+1)\theta\right)$.

To compute the likelihood for a given experiment we can use the function *likelihood*. The input of the method is:

* angle : the angle.
* m_k: the number of times the Grover-like operator will be applied.
* n_k : number of shots.
* h_k: number of positive outcomes

The method returns:
* l_k: likelihood of the angle




In [ ]:
print('m_k: '+str(m_k)+' n_k: '+str(n_k)+' h_k: '+str(h_k))
theta = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,100)
l_k = np.zeros(len(theta))
for i in range(len(theta)):
    l_k[i] = mlae.likelihood(theta[i],m_k,n_k,h_k)

In [ ]:
plt.plot(theta,l_k, '-', label = r"$L(\theta|h_k)$")
plt.axvline(angle_estimation,color = 'r', label = r"$\hat{\theta}$")
plt.xlabel(r'$\theta$')
plt.ylabel('Likelihood')
plt.grid()
plt.legend()

In blue is depicted the likelihood function for different values of $\theta$. In red we have the estimation we made in the previous section.

### 2.4 The Cost Function 

As we see in the last graph, there are multiple values that maximize the likelihood function. If we combine the information from different $(m_k,h_k)$ we can get better estimations.

Therefore we define a combined likelihood as:
$$L(\theta,\mathbf{h}) = \prod_{k = 0}^M l_k(\theta,h_k).$$
$$\mathbf{h} = (h_0, h_1,...,h_M)$$

Instead of dealing with the maximimization problem, we define the equivalent mimimization problem which substituting the combined likelihood with the **cost function** $C$:

$$C(\theta) = -\log\left(L(\theta,\mathbf{h})\right)$$

For computing the **Cost Function** following steps should be followed:

1. First we need to select the different $(m_k,h_k)$ experiments. This is done using the **schedule** that will be a python list of 2 elements:
    * 1st element: list with the number of applications of the grover operator, for example: m_k = [1, 2, 3, 5, 7] 
    * 2nd element list with the number of shots used for each application of the 1st element, for example: n_k = [100, 200, 50, 50, 100]
    * In the before examples the schedule will be: schedule =[m_k, n_k] = [[1, 2, 3, 5, 7], [100, 200, 50, 50, 100]]
2. For each pair ($m_k$, $n_k$) of the schedule run the *run_step* method and get the correspondient $h_k$. This will be done by the *run_schedule* method of the class     
3. Using ($m_k$, $n_k$, $h_k$) we can compute the total cost function for different $\theta \in [0, \frac{\pi}{2}]$ using the *cost_function* static method.



#### 2.4.1 Configure the Schedule

As we said, we want to combine the information of different experiments. In general each experiment can be characterised by the number of aplications of the grover oracle $m_k$ and the number of shots $n_k$. A list of both is what we call a **schedule**.

The schedule can be given when the class **MLAE** using the key *schedule*. 

In [ ]:
m_k = [0,1,2,4,8,16]
n_k = [10]*len(m_k)
schedule = [m_k,n_k]

target = [0,0,1]
index = [0,1,2]

#Schedule not provide
mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': schedule,
    'mcz_qlm': True    
}
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

print('Schedule: \n', mlae.schedule)


We don't need to define the schedule at initialization. In this case following default schedule will be loaded:

* $m_k$ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
* $n_k$ = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

Additionally, we can define the schedule after initialization by simply assigning attribute schedule to the schedule we want

In [ ]:
#Schedule not provide
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True     
}
target = [0,0,1]
index = [0,1,2]
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

print('Default Schedule: \n', mlae.schedule)

#New Schedule
m_k = [0,1,2,4,8,16]
n_k = [10]*len(m_k)
schedule = [m_k,n_k]
mlae.schedule = schedule

print('New Schedule: \n', mlae.schedule)

#### 2.4.2 run_schedule

Once we have the schedule properly configured we can use the *run_schedule* method for executing the application of the Grover operator following the schedule order. This method needs the schedule and provide and a array with the different results of $h_k$ for each pair ($m_k$, $n_k$) of the schedule.


In [ ]:
#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True 
}
target = [0,0,1]
index = [0,1,2]
mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

In [ ]:
#Configure the schedule
#applications of Grover operator
m_k = [0, 1, 2, 3, 4, 5]
#Shots for each application of Grover operator
n_k = [100 for i in m_k]
shedule = [m_k, n_k]
#execution of the run_schedule method
h_k = mlae.run_schedule(shedule)
print('h_k: ', h_k)

#### 2.4.3 cost_function

Now we have all the necesary input ($m_k$, $n_k$, $h_k$) for computing the desired **cost function** $C$. We can use the static method of the class **cost_function**. This method receives:

* $\theta$
* $m_k$
* $n_k$
* $h_k$

and computes the corresponding **Cost Function** for angle $\theta$. 

So we can define an array of $\theta$ between $[0, \frac{\pi}{2}]$ and get the **Cost Function** for each value

**Note** 

To avoid problems with angles 0 and $\frac{\pi}{2}$ we use the delta property of the **MLAE** class

In [ ]:
#Posible angles
angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,50)
cost = np.zeros(len(angles))
#Calculate cost function for each angle
for i in range(len(angles)):
    cost[i] = mlae.cost_function(angles[i], m_k, n_k, h_k)

In [ ]:
#We can plot the Cost Function
plt.plot(angles,cost, '-o', label = r"$C(\theta)$")
plt.title('Cost function')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

We are going to use a little trick: We are going to use the *partial* function from *functools* package for creating a new cost_function based on the *cost_function* method of the **MLAE** class. For this new cost_function we are going to provided the obtained $m_k$, $n_k$ and $h_k$ as a fixed values. So we only need to provide the angle to this new function in order to obtain the cost_function and the obtained $m_k$, $n_k$ and $h_k$ will be used as constants

In [ ]:
#Little trick: We are going to use the partial from functools for giving the m_k, n_k and h_k
from functools import partial
#Usign partial we create a cost_function using as base the cost_function method from the MLAE class
#were the m_k, n_k and h_k will be fixed to the values we desire. Now this new cost_function only needs
#the angle for return the cost using the m_k, n_k and h_k we provide previously as constant values
partial_cost_function = partial(mlae.cost_function, m_k = m_k, n_k=n_k, h_k=h_k)

In [ ]:
#Posible angles
angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,50)
partial_cost = np.zeros(len(angles))
#Calculate cost function for each angle
for i in range(len(angles)):
    partial_cost[i] = partial_cost_function(angles[i])

In [ ]:
#We can plot the Cost Function
plt.plot(angles,partial_cost, '-o', label = r"$C(\theta)$")
plt.title('Cost function')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

### 2.5 Optimization of the Cost Function

So far we have the **cost function** $C$:

$$C(\theta) = -\log\left(L(\theta,\mathbf{h})\right)$$

The idea is the is to find the $\theta^*$ that minimizes the cost function:
$$\theta^* = \arg \min_{\theta} C(\theta)$$

With this $\theta^*$ we can compute the desired $a=sin^2(\theta^*)$. 

This can be done following a straightforward way: finding the lower value of the cost_function array and the correspondient angle:

In [ ]:
#straightforward approach
print('Minimum of the Cost Function: ', min(partial_cost))
theta_min = angles[np.argmin(partial_cost)]
print('theta_min: ', theta_min)

In [ ]:
#Complete plot
plt.plot(angles,partial_cost, '-o', label = r"$C(\theta)$")
plt.axvline(theta_min,color = 'r', label = r"$\theta^*$")
plt.title(r'Cost function and $\theta_{min}$')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

So this straightforward aproximation give us a good approximation but depends on the number of $\theta$ we use to compute the cost function. 

A better approach is to use an optimization routine for getting the minimum of the new *cost_function* function.

For example from **scipy optimization** module we can use the **brute** optimization routine in order to do this as can be seen in the following cells:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brute.html


In [ ]:
import scipy.optimize as so

In [ ]:
#We create a function optimizer that receives as input a function of one variable
theta_domain = [(0+mlae.delta, 0.5*np.pi-mlae.delta)]
optimizer = lambda x: so.brute(func=x, ranges=theta_domain, Ns=1000)

In [ ]:
#We use the optimizer define in before cell given as input the new_cost_function
theta_optimized = optimizer(partial_cost_function)
print('theta_optimized: ', theta_optimized)
print('Cost Function at theta_optimized: ', partial_cost_function(theta_optimized))
print('a estimated from theta_optimized: ', np.sin(theta_optimized)**2)
print("Classical result: ",probability[bitfield_to_int(target)])

In [ ]:
#Now we can plot again all the staff
plt.plot(angles,partial_cost, '-o', label = r"$C(\theta)$")
plt.axvline(theta_optimized,color = 'r', label = r"$\theta_{optimized}$")
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

### 2.6 Complete Algorithm execution

**MLAE** class implements the step by step parts explained in the before sections in an automated way by using the *mlae* method. Once the class is created, user can execute mlae method in order to do a complete execution of **MLAE** algorithm. 

The inputs of this method are:

* schedule.
* optimizer: the optimizer should be passed as a **lambda function** which input will be a function on one variable (the angle to optimize).

The otuputs of this method are:

* result: optimizer result
* h_k : list with positive outcomes of each experiment done according the schedule
* cost_function_partial : python function with the cost function where the $m_k$, $n_k$ and the $h_k$ are fixed to the results obtained.

Additionally this method overwrite the **optimizer_time** property of the class that measures the time for the optimization procces.

In [ ]:
target = [0,0,1]
index = [0,1,2]

#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True     
}

mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)


#Schedule configuration
m_k = [0, 1, 2, 3, 4, 5]
n_k = [100 for i in m_k]
shedule = [m_k, n_k]

#Optimizer Configuration
import scipy.optimize as so
#We create a function optimizer that receives as input the cost_function we create before
theta_domain = [(0+mlae.delta, 0.5*np.pi-mlae.delta)]
optimizer = lambda f: so.brute(func=f, ranges=theta_domain, Ns=1000)

In [ ]:
result, h_k, partial_cost_funtion = mlae.mlae(shedule, optimizer)

In [ ]:
print(result)

In [ ]:
print(h_k)

In [ ]:
print('Optimization time: ', mlae.optimizer_time)

In [ ]:
#We can use the returned partial_cost_function to do some graphs

angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,50)
cost_partial = partial_cost_funtion(angles)
theta_optimized = result[0]

plt.plot(angles, cost_partial, '-o', label = r"$C(\theta)$")
plt.axvline(theta_optimized, color = 'r', label = r"$\theta_{optimized}$")
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

#### The optimizer

To find the angle that mimimizes the cost function we need to use a minimization algorithm. When the class is creted the optimzer can be provided using the key *optimizer* and can be access with property *optimizer*. By default, *optimizer* property will be initialized to the **brute** optimization algorithm from *scipy.optimize*.

Additionally, we can define the optimizer after initialization by simply assigning attribute *optimizer* to the optimizer we want.

**NOTE**

The optimizer that we have to pass to the class needs to be a function of just one variable: the angle. This is straightforward using lambda functions. 


In this case, we will do an example with differential-evolution algorithm from the scipy library.

In [ ]:
import scipy.optimize as so

The *differential evolution* algorithm just needs as input the function to be minimized and the bounds where the minimization is done.

In [ ]:
bounds = [[0.+mlae.delta,np.pi/2-mlae.delta]]
differential_evolution = lambda f: so.differential_evolution(f,bounds = bounds)

In [ ]:
result, h_k, new_cost_function = mlae.mlae(schedule, differential_evolution)

Remember that, the result of the function optimize is the result given by the optimizator. In this case, the function differential evolution gives as output an object. The result of the minimization is stored in the attribute *x*.

In [ ]:
print('Optimization time: ', mlae.optimizer_time)

In [ ]:
angle = result.x
print("Quantum result: ",np.sin(angle)**2)
print("Classical result: ",probability[bitfield_to_int(target)])

### BE AWARE!!!

**Brief comment on the optmizer**. As we saw before, the cost function it is not convex, i.e. it has multiple local minima. If we pass a local optimizer such as *Nelder-Mead*, we won't get a good result. Next we give an example.

In [ ]:
bounds = [[0.+mlae.delta,np.pi/2-mlae.delta]]
#Initial angle value
x0 = [0.8]
nelder_mead = lambda f: so.minimize(f, x0 = x0, bounds = bounds, method = "Nelder-Mead")

In [ ]:
#Remember: if uses other optimizer  than default one the use optimize method instead of run one
mlae.optimizer = nelder_mead
result, h_k, new_cost_function = mlae.mlae(schedule, nelder_mead)

In [ ]:
angles = np.linspace(0++mlae.delta,np.pi/2-mlae.delta,100)
cost = np.zeros(len(angles))
for i in range(len(angles)):
    cost[i] = new_cost_function(angles[i])

In [ ]:
print('Nealder-Meald result: ', result.x)
print('Classical result: ', np.arcsin(probability[bitfield_to_int(target)]**0.5))

In [ ]:
plt.plot(angles,cost,'-o', label = r"$C(\theta)$")
plt.axvline(result.x,color = 'r',label = r"$\theta^*$")
plt.axvline(x0[0],color = 'g',label = r"$\theta_{initial}$")
plt.axvline(angles[cost.argmin()],color = 'b',label = r"$\arg \min_{\theta} C(\theta)}$")
plt.title('Nealder-Mead optimization result')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

In [ ]:
print('Optimization time: ', mlae.optimizer_time)

As can be seen in the graph the *Nealder-Mead* has a bad initialization and optimizer and get stack in a local minimum.

## 3. Sumarize and how to use

In section 2 the complete algorithm in a detailed step by step was explained. The methods shown in this section were shown only for pedagogical purpouses but users **SHOULD NOT** use them.

Users should only use one of the following two methods:

* **mlae** method.
* **run** method.

First, given an oracle user should create the class:

In [ ]:
target = [0,0,1]
index = [0,1,2]

#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True    
}

mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)


### 3.1 *mlae* method

This method was explained in *Section 2.6*. User should provide an scheduler and a optimizer. User can use the defaults in a straightforward way:

In [ ]:
#Default Schedule, default optimizer
result, h_k, partial_cost_function = mlae.mlae(mlae.schedule, mlae.optimizer)

In [ ]:
print("Quantum result theta: ",result)
print("Quantum result a: ",np.sin(result**2))
print('Classical result: ', probability[bitfield_to_int(target)])
print('Test OK: ',abs(np.sin(result**2)-probability[bitfield_to_int(target)]) < 0.005)

In [ ]:
print('Optimizer time: ',mlae.optimizer_time)

### 3.2 *run* method

Additionally, user can configure all the properties of th **MLAE** class, and use directly the *run* method. This method execute the **mlae** method using the atributes of the *MLAE* class (*schedule* and *optimizer*) and return the desired $a=sin^2(\theta^*)$ value. Additionally, the *run* method populates following properties:

* *h_k*: positive outcomes for the schedule used.
* *theta*: estimated angle from a complete *^*MLAE** algorithm execution.
* *ae* amplitude estimation looked parameter extracted from *theta* property: $a=sin^2(\theta^*)$.
* *partial_cost_function*: this is the cost_function where the $m_k$, $n_k$ and $h_k$ variables are fixed to the correpondient values of the used schedule and the obtained $m_k$ values. Only angle $\theta$ can de given to this function.
* *run_time*: this is the elapsed time of a complete *run* method

**WARNING**

*run* method was implemented for using the **brute** optimization algorithm from **scipy.optimize** (i.e. the default optimizer of the **MLAE** class). If user wants to use other optimizer algorithm the method to use **should be** the *mlae* one!!!

In [ ]:
target = [0,0,1]
index = [0,1,2]

m_k = [1, 2, 4, 8, 16]
n_k = [100 for i in m_k]

schedule = [m_k, n_k]


#Creating the class
mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': schedule,
    'mcz_qlm': False 
}

mlae = MLAE(
    oracle,
    target = target,
    index = index, 
    **mlae_dict
)

estimated_a = mlae.run()

In [ ]:
print('estimated_a: ', estimated_a)
#populated properties
print('mlae.h_k: ', mlae.h_k)
print('mlae.theta: ', mlae.theta)
print('mlae.a: ', mlae.ae)

In [ ]:
angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,100)
#using partial_cost_function attribute

plt.plot(angles, mlae.partial_cost_function(angles), '-o', label = r"$C(\theta)$")
plt.axvline(mlae.theta, color = 'r', label = r"$\theta^*$")
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")
plt.legend()

In [ ]:
print("Quantum result a: ", estimated_a)
print('Classical result: ', probability[bitfield_to_int(target)])
print('Test OK: ',abs(estimated_a-probability[bitfield_to_int(target)]) < 0.005)

In [ ]:
#Elapsed times for different Q applications
mlae.time_pdf

In [ ]:
print('Optimizer time: ',mlae.optimizer_time)

In [ ]:
print('Elapsed time for run  method: ',mlae.run_time)

In [ ]:
#For draw the used oracle
grover = mlae._grover_oracle
%qatdisplay grover --depth 3